# Örnek Çalışma 
Bankadaki müşteri profillerinden kişinin bankadan ayrılıp ayrılmayacağına bakacaz

# Kütüphaneleri dahil ediyoruz

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.11.1'

## Part 1 - Data Preprocessing

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values #kredi skorunda tahmini maaşına kadar olanı alıyoruz
y = dataset.iloc[:, -1].values #son sütun

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data
Label Encoding the "Gender" column

In [13]:
from sklearn.preprocessing import LabelEncoder #2 cinsiyet olduğundan 0-1 olarak encod ediyoruz
le = LabelEncoder()#sıralı bir anlam tayatır onehot
X[:, 2] = le.fit_transform(X[:, 2])

In [14]:
print(X)

[[1.0 0.0 0 ... 1 1 101348.88]
 [0.0 0.0 1 ... 0 1 112542.58]
 [1.0 0.0 0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0 ... 0 1 42085.58]
 [0.0 1.0 0 ... 1 0 92888.52]
 [1.0 0.0 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [16]:
print(X)

[[1.0 0.0 1.0 ... 1 1 101348.88]
 [1.0 0.0 0.0 ... 0 1 112542.58]
 [1.0 0.0 1.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 1.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 1.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling önemli derin öğrenme için

In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN
### Initializing the ANN

In [20]:
ann = tf.keras.models.Sequential() #sıralı katmanlı sinir ağı kuruyoruz

### Adding the input layer and the first hidden layer

In [21]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [22]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [23]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # 0 veya 1

## Part 3 - Training the ANN
### Compiling the ANN

In [26]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) #adam gd burada

### Training the ANN on the Training set

In [27]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.6317 - accuracy: 0.6870
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4856 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4429 - accuracy: 0.8010
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4298 - accuracy: 0.8100
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4241 - accuracy: 0.8121
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4208 - accuracy: 0.8150
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4183 - accuracy: 0.8149
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4160 - accuracy: 0.8173
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4140 - accuracy: 0.8177
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4116 - accura

## Part 4 - Making the predictions and evaluating the model
### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

In [41]:
print(X[0])
print(len(X[0]))
print(sc.transform([[1, 0, 1, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))
print(ann.predict(sc.transform([[1, 0, 0, 1, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5) 
#0.5 üstü ise true altı ise false verecek tahmin için

[1.0 0.0 1.0 0 619 0 42 2 0.0 1 1 1 101348.88]
13
[[ 0.5698444  -0.5698444   0.98560362 -0.57369368 -0.52111599  0.91601335
   0.10961719 -0.68538967 -0.2569057   0.8095029   0.64259497  0.9687384
  -0.87203322]]
1/1 [==============================] - 0s 31ms/step
[[False]]


Bu nedenle, ANN modelimiz bu müşterinin bankada kalacağını öngörüyor!

**Önemli not 1:** Tüm özellik değerlerinin çift köşeli parantez içinde girildiğine dikkat edin. Bunun nedeni, "tahmin" yönteminin girdilerinin biçimi olarak her zaman 2 boyutlu bir dizi beklemesidir. Değerlerimizi çift köşeli parantez içine almak, girdiyi tam olarak 2 boyutlu bir dizi haline getirir.

**Önemli not 2:** Ayrıca, "Fransa" ülkesinin son sütunda bir dize olarak değil, ilk üç sütunda "1, 0, 0" olarak girildiğine dikkat edin. Bunun nedeni, tahmin yönteminin elbette eyaletin tek sıcak kodlanmış değerlerini beklemesi ve X özellik matrisinin ilk satırında gördüğümüz gibi, "Fransa"nın "1, 0, 0" olarak kodlanmış olmasıdır. Bu değerleri ilk üç sütuna dahil etmeye dikkat edin, çünkü kukla değişkenler her zaman ilk sütunlarda oluşturulur.

In [44]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
#sol tarafa gerçek sonuçkar sağ tarafı tahmin edilenler

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [48]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)
#%86 doğru kontrol ediyor
#1523 müşterinin kalacağını doğru tahmin ediyor
#203 müşterinin kalmayacağını doğru tahmin ediyor
#72 kişiyi ayrılacak diye yanlış hesaplıyor
#202 müşteriyi kalacak diyerek yanlış hesaplıyor

[[1523   72]
 [ 202  203]]


0.863

### 